### 20184645 김영찬
### 기상청 데이터 응용 예

In [32]:
# 기상청 현재 날씨 페이지 읽기
import urllib.request
from bs4 import BeautifulSoup

page = urllib.request.urlopen("http://www.weather.go.kr/weather/observation/currentweather.jsp").read()
soup = BeautifulSoup(page, 'html.parser')

In [33]:
# 데이터 베이스 연결
import sqlite3
conn = sqlite3.connect('Weather.db')
# 얀걀 객체의 row_factory 속성을 sqlite3.Row클래스로 지정
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [34]:
# 햔재 시간 입력태그 검색
current_time = soup.find('input',{'class':'time'})
#날짜 시간 분리 후 sqlite3 향식의 날짜 시간 문자열로 변환
dt = current_time['value'].split('.')
year = dt[0]
month = dt[1]
date = dt[2]
time = dt[3]
date_time = year+"-"+month+"-"+date+" "+time
print(date_time)

2021-10-06 13:00


In [35]:
# 현재 날씨 테이블 검색
table = soup.find('table',{'class':'table_develop3'})
                                    # 테이블 시작, 검색
data = []                           # 데이터를 저장할 리스트 생성

In [36]:
# 재실행시 마다 중복된 데이터를 계속 삽입하기 떄문에 테이블을 초기화 하기위해 작성
cursor.execute('delete from CityWeather')

#모든 <tr> 태그를 검색 각도시의 데이터를 크롤링하고 데이터베이스에 삽입
for tr in table.find_all('tr'):
    tds = list(tr.find_all('td'))
    for td in tds:
        if td.find('a'):
            city = td.find('a').text
            temperature = tds[5].text
            humidity = tds[9].text
            data.append([city,temperature,humidity])
            # 레코드 삽입 sql
            cursor.execute('''
            insert into CityWeather (city, temperature, humidity, dtime) values(?,?,?,?)
            ''',  (city, temperature, humidity, date_time))
conn.commit() # 데이터 베이스 내보내기

In [37]:
# 온도가 27도 이상인 도시 조회
cursor.execute('select * from CityWeather where temperature > 27')
rows=cursor.fetchall()

# 습도가 80 이상인 도시 조회
cursor.execute('select * from CityWeather where humidity > 80')
sql1=cursor.fetchall()

# 도시의 이름만 모두 조회
cursor.execute('select city from CityWeather')
sql2=cursor.fetchall()

# 도시의 이름으 수원인 자료 조회
cursor.execute('select * from CityWeather where city="수원"')
sql3=cursor.fetchall()


for row in rows:
    print("도시 : %s, 온도: %s, 습도: %s, 시간: %s"%
          (row['city'],row['temperature'],row['humidity'],row['dtime']))

print('-----------------------------------------------------------------------')

for sql in sql1:
    print("도시 : %s, 온도: %s, 습도: %s, 시간: %s"%
          (sql['city'],sql['temperature'],sql['humidity'],sql['dtime']))
    
print('-----------------------------------------------------------------------')

for sql in sql2:
    print("도시 : %s"%
          (sql['city']))

print('-----------------------------------------------------------------------')

for sql in sql3:
    print("도시 : %s, 온도: %s, 습도: %s, 시간: %s"%
          (sql['city'],sql['temperature'],sql['humidity'],sql['dtime']))
  
conn.close()

도시 : 강진군, 온도: 28.0, 습도: 56, 시간: 2021-10-06 13:00
도시 : 고산, 온도: 27.4, 습도: 71, 시간: 2021-10-06 13:00
도시 : 고창, 온도: 28.6, 습도: 49, 시간: 2021-10-06 13:00
도시 : 고창군, 온도: 28.2, 습도: 46, 시간: 2021-10-06 13:00
도시 : 광양시, 온도: 28.0, 습도: 60, 시간: 2021-10-06 13:00
도시 : 광주, 온도: 28.2, 습도: 51, 시간: 2021-10-06 13:00
도시 : 김해시, 온도: 27.1, 습도: 65, 시간: 2021-10-06 13:00
도시 : 남원, 온도: 27.5, 습도: 46, 시간: 2021-10-06 13:00
도시 : 대전, 온도: 27.9, 습도: 59, 시간: 2021-10-06 13:00
도시 : 목포, 온도: 27.2, 습도: 60, 시간: 2021-10-06 13:00
도시 : 밀양, 온도: 28.2, 습도: 46, 시간: 2021-10-06 13:00
도시 : 보성군, 온도: 27.5, 습도: 63, 시간: 2021-10-06 13:00
도시 : 부안, 온도: 27.3, 습도: 43, 시간: 2021-10-06 13:00
도시 : 산청, 온도: 27.3, 습도: 55, 시간: 2021-10-06 13:00
도시 : 서귀포, 온도: 27.2, 습도: 70, 시간: 2021-10-06 13:00
도시 : 순창군, 온도: 27.2, 습도: 53, 시간: 2021-10-06 13:00
도시 : 순천, 온도: 27.2, 습도: 58, 시간: 2021-10-06 13:00
도시 : 양산시, 온도: 28.8, 습도: 53, 시간: 2021-10-06 13:00
도시 : 영광군, 온도: 27.2, 습도: 51, 시간: 2021-10-06 13:00
도시 : 완도, 온도: 27.7, 습도: 66, 시간: 2021-10-06 13:00
도시 : 의령군, 온도: 29.1, 습도: 57, 시간: